# Matrix-model Intermediate Example
This example aims to highlight the follow:
* build a partial matrix of distances/times between a collection of sources and destinations

## Requirements
This example assumes you've configured an api-key with the required services enabled (see the [portal](portal.icepack.ai) for configuration details) and that you're familiar with loading and working with protobuf models.

## Data
We're going to load a small sample data set (just 6 locations) and generate a subset of distances between them.


In [ ]:
import pandas
df = pandas.read_csv('../../sample_data/publist.csv').head(6)
print(df.head())

## Model helper
There are several common code-snippets which are useful when working with the icepack models. We use the same standard for post/get queries for all models. We're going to use the api helper in this context to help manage the calls to the api.

We're going to:
1. load the helper
2. instantiate a matrix model request wiht a partial matrix request (two sources to four destinations)
3. populate the request
4. post the request (helper class)
5. get the response (helper class)

### Build the request

In [ ]:
exec(open('apiHelper.py').read()) # import some api-helper classes which we've written for you.

api = apiHelper(modelType="matrix-vyv95n7wchpl") # set the model type to the matrix model

mr = matrix_vyv95n7wchpl_pb2.MatrixRequest()

for index, row in df.iterrows():
    l = matrix_vyv95n7wchpl_pb2.Location()
    l.id = row.id
    l.geocode.longitude = row.X
    l.geocode.latitude = row.Y
    mr.locations.append(l)                    # add the location definition to the list of locations
    if index < 2:
        mr.sources.append(row.id)             # add the first two locations as sources
    else:
        mr.destinations.append(row.id)        # and the remaining four locations as destinations

mr.distanceUnit = 0
mr.durationUnit = 1
print(mr)


We would expect this to create a 2x4 matrix. locations 1:2 to locations 3:6 which will returns a total of 8 (2 * 4) elements.
### Run the model


In [ ]:
reqId = api.Post(mr)  # send the request
print(reqId) # if you want to see what the guid looks like.
sol = api.Get(reqId)  # get the response


In [ ]:
print(len(sol.elements)) # the number of elements returned by the api - which is 8 - Hooray!

### Some notes
You'll see we get 8 elements in the response from the api - which was expected (2x4)

We can tabulate the response by calling the `tabulate` method we've bundled here.

In [ ]:
matrix = tabulate(mr, sol)
matrix


This matrix is expressed in long-form. Nothing wrong with that, but you might prefer to see it in wide form. Pandas has a nice function for this.


In [ ]:
import numpy as np
dm = pandas.crosstab(index= matrix['fromId'], 
                columns=matrix['toId'], 
                values = matrix['distance'], aggfunc = np.sum).fillna(0)

# this creates a distance matrix 
dm

In [ ]:
tm = pandas.crosstab(index= matrix['fromId'], 
                columns=matrix['toId'], 
                values = matrix['duration'], aggfunc = np.sum).fillna(0)
#and this creates a time matrix
tm

### What next?
This is about as complicated as a matrix request gets luckily :-) 

_It's worth noting that we limit the matrix size to 160,000 elements as a maximum request (so a 400x400 or thereabouts)._